In [1]:
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import numpy as np
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor
from tsfresh.feature_extraction import EfficientFCParameters
import os

with h5py.File('train.h5', 'r') as f:
    X = f['x'][:]
    Y = f['y'][:]

print(X.shape)
print(Y.shape)

# Convierte X a un DataFrame de pandas
df_X = pd.DataFrame(X.reshape(-1, X.shape[-1]))

# Convierte Y a un DataFrame de pandas
df_Y = pd.DataFrame(Y, columns=['Target'])

# Une los dos DataFrames
df = pd.concat([df_X, df_Y], axis=1)

# print(df)

num_trials = 18530

df_tsfresh = pd.DataFrame({
    'id': np.repeat(df.index, num_trials),
    'time': np.tile(np.arange(num_trials), len(df)),
    'value': df.drop(columns='Target').values.flatten()
})


#use test.h5 and predict the target and save it in a file
with h5py.File('test.h5', 'r') as f:
    XX = f['x'][:]

# Convierte X a un DataFrame de pandas
df_X_test = pd.DataFrame(XX.reshape(-1, X.shape[-1]))

df_tsfresh_test = pd.DataFrame({
    'id': np.repeat(df_X_test.index, num_trials),
    'time': np.tile(np.arange(num_trials), len(df_X_test)),
    'value': df_X_test.values.flatten()
})

(204, 1, 18530)
(204,)


In [2]:
from tsfresh.utilities.distribution import MultiprocessingDistributor

import os

custom_settings = {
    "absolute_sum_of_changes": None,
    "mean": None,
    "variance": None,
    "fft_aggregated": [
        {"aggtype": "centroid"},
        {"aggtype": "variance"},
        {"aggtype": "skew"},
        {"aggtype": "kurtosis"}
    ],
    "permutation_entropy": [{"tau": 1, "dimension": 3}],
    "fractal_dimension": [{"method": "katz"}],
    "fractal_dimension": [{"method": "petrosian"}]
}

num_cores = os.cpu_count()
Distributor = MultiprocessingDistributor(n_workers=num_cores , disable_progressbar=False ,
                                         progressbar_title="Extraccion de features")



features = extract_features(df_tsfresh, column_id='id', column_sort='time', distributor=Distributor, default_fc_parameters=custom_settings)

Extraccion de features:   0%|          | 0/102 [00:06<?, ?it/s]


MemoryError: Unable to allocate 5.12 GiB for an array with shape (18529, 18529, 2) and data type float64